In [1]:
import pandas as pd
import numpy as np
from scipy.stats import percentileofscore
import seaborn as sns
import matplotlib.pyplot as plt
from vol_data import *
from surface import *
import pickle
import pprint
import time
import ipywidgets as widgets
from IPython.display import display, HTML
import os
from collections import OrderedDict

#### Function

In [2]:
def get_vol_moneyness(udl_list, matu_list, moneyness_list, start_date, end_date):
    """
    Generate random volatility moneyness data.
    """
    date_range = pd.date_range(start=start_date, end=end_date, freq='B')
    data = {
        (udl, matu, mon): np.random.rand(len(date_range)) * 100
        for udl in udl_list
        for matu in matu_list
        for mon in moneyness_list
    }
    df = pd.DataFrame(data, index=date_range)
    df.columns = pd.MultiIndex.from_tuples(df.columns, names=['udl', 'matu', 'moneyness'])
    return df

def get_vol_delta(udl_list, matu_list, delta_list, start_date, end_date):
    """
    Generate random volatility delta data.
    """
    date_range = pd.date_range(start=start_date, end=end_date, freq='B')
    data = {
        (udl, matu, delta): np.random.rand(len(date_range)) * 100
        for udl in udl_list
        for matu in matu_list
        for delta in delta_list
    }
    df = pd.DataFrame(data, index=date_range)
    df.columns = pd.MultiIndex.from_tuples(df.columns, names=['udl', 'matu', 'delta'])
    return df
    
def generate_table(udl_list, matu_list, moneyness_list, delta_list, start_date, end_date):
    """
    Generate random volatility data for moneyness and delta in a single step and create a structured DataFrame.

    Args:
        udl_list (list): List of underlying assets.
        matu_list (list): List of maturities.
        moneyness_list (list): List of moneyness levels.
        delta_list (list): List of delta levels.
        start_date (str): Start date for the data.
        end_date (str): End date for the data.

    Returns:
        DataFrame: DataFrame with combined moneyness and delta data, indexed by date with MultiIndex columns.
    """
    try:
        date_range = pd.date_range(start=start_date, end=end_date, freq='B')

        # Generate moneyness and delta data
        df_vol_moneyness = get_vol_moneyness(udl_list, matu_list, moneyness_list, start_date, end_date)
        df_vol_delta = get_vol_delta(udl_list, matu_list, delta_list, start_date, end_date)

        data = {}

        # Combine moneyness data
        for udl in udl_list:
            for matu in matu_list:
                for mon in moneyness_list:
                    key = (udl, matu, mon)
                    data[(udl, 'IV', matu, mon)] = df_vol_moneyness[key]

        # Combine delta data
        for udl in udl_list:
            for matu in matu_list:
                for delta in delta_list:
                    key = (udl, matu, delta)
                    data[(udl, 'IVFD', matu, delta)] = df_vol_delta[key]

        df = pd.DataFrame(data, index=date_range)
        df.columns = pd.MultiIndex.from_tuples(df.columns, names=['udl', 'param', 'matu', 'value'])
        df.index.name = 'Date'
        df.ffill(inplace=True)

        return df
    except Exception as e:
        print(f"An error occurred during table generation: {e}")
        return pd.DataFrame()

def df_to_nested_dict(df):
    nested_dict = {}
    for date, row in df.iterrows():
        date_str = date.strftime('%Y-%m-%d')
        nested_dict[date_str] = {}
        for col, val in row.items():
            udl, param, matu, value = col
            nested_dict[date_str].setdefault(udl, {}).setdefault(param, {}).setdefault(matu, {})[value] = val
    return nested_dict

def sanity_check_surface(df, computed_surface, udl, date, moneyness_levels, surface_type):
    date_range = pd.date_range(start=df.index.min(), end=date, freq='B')
    df_filtered = df.loc[date_range]

    for matu in computed_surface.index:
        for mon in moneyness_levels:
            col_name = f'{mon}'
            if col_name in computed_surface.columns:
                values = df_filtered.xs((udl, 'IV', matu, mon), level=['udl', 'param', 'matu', 'value'], axis=1).dropna().values.flatten()
                if len(values) > 0:
                    if surface_type == 'percentile':
                        expected_value = percentileofscore(values, df.loc[date, (udl, 'IV', matu, mon)], kind='mean')
                    elif surface_type == 'zscore':
                        mean = values.mean()
                        std = values.std(ddof=1)
                        expected_value = (df.loc[date, (udl, 'IV', matu, mon)] - mean) / std if std > 0 else np.nan
                    else:
                        continue
                    computed_value = computed_surface.at[matu, col_name]
                    if not np.isclose(expected_value, computed_value, atol=1e-2, equal_nan=True):
                        print(f"Mismatch at Maturity: {matu}, Moneyness: {mon}. Expected {surface_type.capitalize()}: {expected_value}, Computed {surface_type.capitalize()}: {computed_value}")
                        return False
    return True
    
    
def calculate_percentile_rank_surface(nested_dict, udl, date, moneyness_levels):
    try:
        date_str = date.strftime('%Y-%m-%d')
        percentile_rank_surface = pd.DataFrame(index=moneyness_levels, columns=nested_dict[date_str][udl]['IV'].keys())

        for matu in percentile_rank_surface.columns:
            for mon in moneyness_levels:
                try:
                    values = []
                    for past_date in nested_dict:
                        if udl in nested_dict[past_date] and 'IV' in nested_dict[past_date][udl] and matu in nested_dict[past_date][udl]['IV'] and mon in nested_dict[past_date][udl]['IV'][matu]:
                            values.append(nested_dict[past_date][udl]['IV'][matu][mon])
                    if len(values) > 0:
                        current_value = nested_dict[date_str][udl]['IV'][matu][mon]
                        percentile = percentileofscore(values, current_value, kind='mean')
                        percentile_rank_surface.at[mon, matu] = percentile
                    else:
                        percentile_rank_surface.at[mon, matu] = np.nan
                except KeyError:
                    percentile_rank_surface.at[mon, matu] = np.nan

        return percentile_rank_surface.T
    except Exception as e:
        print(f"An error occurred in calculate_percentile_rank_surface: {e}")
        return pd.DataFrame()

def calculate_zscore_surface(nested_dict, udl, date, moneyness_levels):
    try:
        date_str = date.strftime('%Y-%m-%d')
        zscore_surface = pd.DataFrame(index=moneyness_levels, columns=nested_dict[date_str][udl]['IV'].keys())

        for matu in zscore_surface.columns:
            for mon in moneyness_levels:
                try:
                    values = []
                    for past_date in nested_dict:
                        if udl in nested_dict[past_date] and 'IV' in nested_dict[past_date][udl] and matu in nested_dict[past_date][udl]['IV'] and mon in nested_dict[past_date][udl]['IV'][matu]:
                            values.append(nested_dict[past_date][udl]['IV'][matu][mon])
                    if len(values) > 0:
                        mean = np.mean(values)
                        std = np.std(values, ddof=1)
                        current_value = nested_dict[date_str][udl]['IV'][matu][mon]
                        if std > 0:
                            zscore = (current_value - mean) / std
                            zscore_surface.at[mon, matu] = zscore
                        else:
                            zscore_surface.at[mon, matu] = np.nan
                    else:
                        zscore_surface.at[mon, matu] = np.nan
                except KeyError:
                    zscore_surface.at[mon, matu] = np.nan

        return zscore_surface.T
    except Exception as e:
        print(f"An error occurred in calculate_zscore_surface: {e}")
        return pd.DataFrame()

def plot_surface(udl, date, surface_type, start_date=None, end_date=None):
    try:
        date = pd.Timestamp(date)

        if surface_type == 'Level':
            vol_surface = create_vol_surface(nested_dict, date, udl, moneyness_list)
            vol_surface = ensure_numerical(vol_surface)  # Ensure numerical format
            display(style_df(vol_surface, "Volatility Surface"))
        elif surface_type == 'Percentile':
            if start_date and end_date:
                start_date = pd.Timestamp(start_date)
                end_date = pd.Timestamp(end_date)
                if start_date > end_date:
                    print("Start date cannot be after end date.")
                    return
                filtered_dict = {k: v for k, v in nested_dict.items() if start_date <= pd.Timestamp(k) <= end_date}
                percentile_surface = calculate_percentile_rank_surface(filtered_dict, udl, end_date, moneyness_list)
                percentile_surface = ensure_numerical(percentile_surface)  # Ensure numerical format
                title = f"Percentile Surface ({udl}) From: {start_date.strftime('%Y-%m-%d')} to: {end_date.strftime('%Y-%m-%d')}"
                styled_df = style_df(percentile_surface, title)
                display(styled_df)
            else:
                print("Please select start and end dates for Percentile surface.")
        elif surface_type == 'Z-score':
            if start_date and end_date:
                start_date = pd.Timestamp(start_date)
                end_date = pd.Timestamp(end_date)
                if start_date > end_date:
                    print("Start date cannot be after end date.")
                    return
                filtered_dict = {k: v for k, v in nested_dict.items() if start_date <= pd.Timestamp(k) <= end_date}
                zscore_surface = calculate_zscore_surface(filtered_dict, udl, end_date, moneyness_list)
                zscore_surface = ensure_numerical(zscore_surface)  # Ensure numerical format
                title = f"Z-score Surface ({udl}) From: {start_date.strftime('%Y-%m-%d')} to: {end_date.strftime('%Y-%m-%d')}"
                styled_df = style_df(zscore_surface, title)
                display(styled_df)
            else:
                print("Please select start and end dates for Z-score surface.")
        else:
            print("Invalid surface type selected.")
    except KeyError as e:
        print(f"KeyError: {e} - Ensure the selected date range is within the data's date range.")
    except Exception as e:
        print(f"An error occurred: {e}")

def format_and_adjust_column_names(df):
    formatted_columns = []
    for col in df.columns:
        try:
            float_col = float(col)
            if float_col.is_integer():
                formatted_columns.append(f"{int(float_col)}")
            else:
                formatted_columns.append(f"{float_col:.1f}")
        except ValueError:
            formatted_columns.append(str(col))  # Ensure the column name is a string even if it cannot be converted to float
    df.columns = formatted_columns
    df.index = [str(idx) for idx in df.index]  # Ensure the index is formatted as strings
    return df
    
import matplotlib.colors as mcolors
BNPP_colors = [
    (0, 124/256, 177/256), # Blue
    (112/256, 194/256, 122/256), # Green
    (238/256, 48/256, 46/256) # Red
]

BNPP_colors_float = [[c/256 for c in color] for color in BNPP_colors]

GR_cmap = mcolors.LinearSegmentedColormap.from_list("GR_cmap", [BNPP_colors[1], BNPP_colors[2]], N=256)

def style_df(df, caption):
    df = format_and_adjust_column_names(df)  # Format and adjust column names
    cm = sns.light_palette("green", as_cmap=True)
    df_styled = df.style.background_gradient(cmap=GR_cmap).format("{:.1f}").set_table_styles([
        {'selector': 'th', 'props': [('min-width', '90px'), ('max-width', '90px'), ('text-align', 'center')]},
        {'selector': 'td', 'props': [('text-align', 'center')]}
    ]).set_properties(**{'text-align': 'center'})
    
    # Add caption and table attributes
    df_styled = df_styled.set_caption(caption).set_table_attributes('style="width:100%; border-collapse:collapse; border: 1px solid black;"')
    
    return df_styled
    
def ensure_numerical(df):
    return df.apply(pd.to_numeric, errors='coerce')

# Function to toggle date widgets visibility
def toggle_date_widgets(surface_type):
    if surface_type in ['Percentile', 'Z-score']:
        start_date_widget.layout.display = 'block'
        end_date_widget.layout.display = 'block'
        date_widget.layout.display = 'none'
    else:
        start_date_widget.layout.display = 'none'
        end_date_widget.layout.display = 'none'
        date_widget.layout.display = 'block'

# Function to create a volatility surface
def create_vol_surface(nested_dict, date, udl, moneyness_levels):
    date_str = date if isinstance(date, str) else date.strftime('%Y-%m-%d')
    if date_str not in nested_dict:
        raise ValueError(f"Date {date_str} not found in data.")

    vol_surface = pd.DataFrame(index=moneyness_levels)
    for matu, moneyness_data in nested_dict[date_str][udl]['IV'].items():
        vol_surface[matu] = [moneyness_data.get(moneyness, 
                                                np.nan) for moneyness in moneyness_levels]
    vol_surface = vol_surface.T
    vol_surface.columns = [f'{mon}' for mon in vol_surface.columns]
    vol_surface.index.name = 'Maturity'
    vol_surface = vol_surface.map(lambda x: round(x, 2) if pd.notna(x) else np.nan)
    return vol_surface

### Inputs

In [3]:
start_date = '2024-01-01'
end_date = '2024-05-28'
date = '2024-05-21'

udl_list = ['JP_NKY', 'DE_DAX', 'GB_FTSE100', 'CH_SMI', 'IT_FTMIB', 'ES_IBEX', 'US_SPX', 'EU_STOXX50E', 'EU_SX7E', 'EU_SX7P',
            'EU_SXDP', 'US_KO', 'US_MCD', 'US_KOMO', 'EU_SXPP', 'EU_SOXP', 'HK_HSI']

udl =  'EU_STOXX50E'

matu_list = [1, 3, 6, 9, 12, 18, 24, 36]
moneyness_list = [120, 110, 105, 102.5, 100, 97.5, 95, 90, 80]
delta_list = [5, 10, 15, 20, 25, 35, 50, 65, 75, 90, 95]

### Generate data to modify

In [4]:
# Read dict
path = "vol_surf.pickle"
with open(path, 'rb') as handle:
    nested_dict = pickle.load(handle)

### APP

In [5]:
# Widget definitions
udl_widget = widgets.Dropdown(
    options=udl_list,
    value='EU_STOXX50E',
    description='UDL:',
    disabled=False,
)

date_widget = widgets.DatePicker(
    description='Date',
    value=pd.to_datetime(end_date),
    disabled=False
)

start_date_widget = widgets.DatePicker(
    description='Start Date',
    value=pd.to_datetime(start_date),
    disabled=False
)

end_date_widget = widgets.DatePicker(
    description='End Date',
    value=pd.to_datetime(end_date),
    disabled=False
)

surface_type_widget = widgets.Dropdown(
    options=['Level', 'Percentile', 'Z-score'],
    value='Level',
    description='Type:',
    disabled=False,
)
# Widgets
udl_widget = widgets.Dropdown(options=udl_list, description='UDL:')
surface_type_widget = widgets.Dropdown(options=['Level', 'Percentile', 'Z-score'], description='Type:')
date_widget = widgets.DatePicker(description='Date', value=pd.to_datetime('2024-05-28'))
start_date_widget = widgets.DatePicker(description='Start Date', value=pd.to_datetime('2024-01-01'))
end_date_widget = widgets.DatePicker(description='End Date', value=pd.to_datetime('2024-05-27'))

# Observe changes in surface type to toggle date widgets
surface_type_widget.observe(lambda change: toggle_date_widgets(change['new']), names='value')

# Create interactive output
output = widgets.interactive_output(plot_surface, {
    'udl': udl_widget, 
    'date': date_widget,
    'surface_type': surface_type_widget,
    'start_date': start_date_widget,
    'end_date': end_date_widget
})


# Layout for widgets
left_box = widgets.VBox([udl_widget, surface_type_widget], layout=widgets.Layout(margin='10px'))
right_box = widgets.VBox([start_date_widget, end_date_widget, date_widget], layout=widgets.Layout(margin='10px'))
top_box = widgets.HBox([left_box, right_box], layout=widgets.Layout(justify_content='space-between', align_items='center', margin='10px'))
main_box = widgets.VBox([top_box, output])

# Set initial visibility
toggle_date_widgets(surface_type_widget.value)

# Display the widgets and output
display(main_box)

### Scheduled

In [6]:
#!pip install schedule
import warnings

# Suppress the specific FutureWarning
warnings.simplefilter(action='ignore', category=FutureWarning)
import pandas as pd
import pickle
import schedule
import time
import threading
from datetime import datetime
from vol_data import *
from surface import *

In [7]:
# Define your parameters
spe_time = "22:48"

start_date = '2024-01-01'
end_date = '2024-05-28'

udl_list = ['JP_NKY', 'DE_DAX', 'GB_FTSE100', 'CH_SMI', 'IT_FTMIB', 'ES_IBEX', 'US_SPX', 'EU_STOXX50E', 'EU_SX7E', 'EU_SX7P',
            'EU_SXDP', 'US_KO', 'US_MCD', 'US_KOMO', 'EU_SXPP', 'EU_SOXP', 'HK_HSI']

matu_list = [1, 3, 6, 9, 12, 18, 24, 36]
moneyness_list = [120, 110, 105, 102.5, 100, 97.5, 95, 90, 80]
delta_list = [5, 10, 15, 20, 25, 35, 50, 65, 75, 90, 95]

path = "vol_surf.pickle"  # Path to save the pickle file

In [8]:
def get_vol_moneyness(udl_list, matu_list, moneyness_list, start_date, end_date):
    """
    Generate random volatility moneyness data.
    """
    date_range = pd.date_range(start=start_date, end=end_date, freq='B')
    data = {
        (udl, matu, mon): np.random.rand(len(date_range)) * 100
        for udl in udl_list
        for matu in matu_list
        for mon in moneyness_list
    }
    df = pd.DataFrame(data, index=date_range)
    df.columns = pd.MultiIndex.from_tuples(df.columns, names=['udl', 'matu', 'moneyness'])
    return df

def get_vol_delta(udl_list, matu_list, delta_list, start_date, end_date):
    """
    Generate random volatility delta data.
    """
    date_range = pd.date_range(start=start_date, end=end_date, freq='B')
    data = {
        (udl, matu, delta): np.random.rand(len(date_range)) * 100
        for udl in udl_list
        for matu in matu_list
        for delta in delta_list
    }
    df = pd.DataFrame(data, index=date_range)
    df.columns = pd.MultiIndex.from_tuples(df.columns, names=['udl', 'matu', 'delta'])
    return df
    
def generate_table(udl_list, matu_list, moneyness_list, delta_list, start_date, end_date):
    """
    Generate random volatility data for moneyness and delta in a single step and create a structured DataFrame.

    Args:
        udl_list (list): List of underlying assets.
        matu_list (list): List of maturities.
        moneyness_list (list): List of moneyness levels.
        delta_list (list): List of delta levels.
        start_date (str): Start date for the data.
        end_date (str): End date for the data.

    Returns:
        DataFrame: DataFrame with combined moneyness and delta data, indexed by date with MultiIndex columns.
    """
    try:
        date_range = pd.date_range(start=start_date, end=end_date, freq='B')

        # Generate moneyness and delta data
        df_vol_moneyness = get_vol_moneyness(udl_list, matu_list, moneyness_list, start_date, end_date)
        df_vol_delta = get_vol_delta(udl_list, matu_list, delta_list, start_date, end_date)

        data = {}

        # Combine moneyness data
        for udl in udl_list:
            for matu in matu_list:
                for mon in moneyness_list:
                    key = (udl, matu, mon)
                    data[(udl, 'IV', matu, mon)] = df_vol_moneyness[key]

        # Combine delta data
        for udl in udl_list:
            for matu in matu_list:
                for delta in delta_list:
                    key = (udl, matu, delta)
                    data[(udl, 'IVFD', matu, delta)] = df_vol_delta[key]

        df = pd.DataFrame(data, index=date_range)
        df.columns = pd.MultiIndex.from_tuples(df.columns, names=['udl', 'param', 'matu', 'value'])
        df.index.name = 'Date'
        df.ffill(inplace=True)

        return df
    except Exception as e:
        print(f"An error occurred during table generation: {e}")
        return pd.DataFrame()

def df_to_nested_dict(df):
    nested_dict = {}
    for date, row in df.iterrows():
        date_str = date.strftime('%Y-%m-%d')
        nested_dict[date_str] = {}
        for col, val in row.items():
            udl, param, matu, value = col
            nested_dict[date_str].setdefault(udl, {}).setdefault(param, {}).setdefault(matu, {})[value] = val
    return nested_dict

def generate_and_save_vol_surf():
    try:
        # Generate the table
        df = generate_table(udl_list, matu_list, moneyness_list, delta_list, start_date, end_date)

        # Convert DataFrame to nested dictionary
        nested_dict = df_to_nested_dict(df)

        # Save the nested dictionary to a pickle file
        with open(path, 'wb') as handle:
            pickle.dump(nested_dict, handle)

        print(f"{datetime.now()} - Volatility surface data generated and saved successfully.")
    except Exception as e:
        print(f"An error occurred: {e}")

def run_schedule():
    while True:
        schedule.run_pending()
        time.sleep(1)  # Wait a bit before checking again

In [9]:
# Schedule the job to run at midnight every day
schedule.every().day.at(spe_time).do(generate_and_save_vol_surf)

# Start the scheduler in a new thread
scheduler_thread = threading.Thread(target=run_schedule, daemon=True)
scheduler_thread.start()

In [10]:
print("Scheduler started. The task will run every day at: " + spe_time)

Scheduler started. The task will run every day at: 22:48


### DEV

Added/Updated Data Generation Functions: These functions remain the same since we are using the same method to generate random volatility data. No changes were needed here.

get_vol_moneyness
get_vol_delta
generate_table
Adjusted Calculation Functions: These functions were updated to calculate the spread between two UDLs.

calculate_percentile_rank_surface
calculate_zscore_surface
create_vol_surface

In [11]:
def calculate_percentile_rank_surface(nested_dict, udl1, udl2, date, moneyness_levels):
    try:
        date_str = date.strftime('%Y-%m-%d')
        percentile_rank_surface = pd.DataFrame(index=moneyness_levels, columns=nested_dict[date_str][udl1]['IV'].keys())

        for matu in percentile_rank_surface.columns:
            for mon in moneyness_levels:
                try:
                    values = []
                    for past_date in nested_dict:
                        if (udl1 in nested_dict[past_date] and 'IV' in nested_dict[past_date][udl1] and 
                            matu in nested_dict[past_date][udl1]['IV'] and mon in nested_dict[past_date][udl1]['IV'][matu] and
                            udl2 in nested_dict[past_date] and 'IV' in nested_dict[past_date][udl2] and 
                            matu in nested_dict[past_date][udl2]['IV'] and mon in nested_dict[past_date][udl2]['IV'][matu]):
                            spread = nested_dict[past_date][udl1]['IV'][matu][mon] - nested_dict[past_date][udl2]['IV'][matu][mon]
                            values.append(spread)
                    if len(values) > 0:
                        current_value = nested_dict[date_str][udl1]['IV'][matu][mon] - nested_dict[date_str][udl2]['IV'][matu][mon]
                        percentile = percentileofscore(values, current_value, kind='mean')
                        percentile_rank_surface.at[mon, matu] = percentile
                    else:
                        percentile_rank_surface.at[mon, matu] = np.nan
                except KeyError:
                    percentile_rank_surface.at[mon, matu] = np.nan

        return percentile_rank_surface.T
    except Exception as e:
        print(f"An error occurred in calculate_percentile_rank_surface: {e}")
        return pd.DataFrame()

def calculate_zscore_surface(nested_dict, udl1, udl2, date, moneyness_levels):
    try:
        date_str = date.strftime('%Y-%m-%d')
        zscore_surface = pd.DataFrame(index=moneyness_levels, columns=nested_dict[date_str][udl1]['IV'].keys())

        for matu in zscore_surface.columns:
            for mon in moneyness_levels:
                try:
                    values = []
                    for past_date in nested_dict:
                        if (udl1 in nested_dict[past_date] and 'IV' in nested_dict[past_date][udl1] and 
                            matu in nested_dict[past_date][udl1]['IV'] and mon in nested_dict[past_date][udl1]['IV'][matu] and
                            udl2 in nested_dict[past_date] and 'IV' in nested_dict[past_date][udl2] and 
                            matu in nested_dict[past_date][udl2]['IV'] and mon in nested_dict[past_date][udl2]['IV'][matu]):
                            spread = nested_dict[past_date][udl1]['IV'][matu][mon] - nested_dict[past_date][udl2]['IV'][matu][mon]
                            values.append(spread)
                    if len(values) > 0:
                        mean = np.mean(values)
                        std = np.std(values, ddof=1)
                        current_value = nested_dict[date_str][udl1]['IV'][matu][mon] - nested_dict[date_str][udl2]['IV'][matu][mon]
                        if std > 0:
                            zscore = (current_value - mean) / std
                            zscore_surface.at[mon, matu] = zscore
                        else:
                            zscore_surface.at[mon, matu] = np.nan
                    else:
                        zscore_surface.at[mon, matu] = np.nan
                except KeyError:
                    zscore_surface.at[mon, matu] = np.nan

        return zscore_surface.T
    except Exception as e:
        print(f"An error occurred in calculate_zscore_surface: {e}")
        return pd.DataFrame()

def create_vol_surface(nested_dict, date, udl1, udl2, moneyness_levels):
    date_str = date if isinstance(date, str) else date.strftime('%Y-%m-%d')
    if date_str not in nested_dict:
        raise ValueError(f"Date {date_str} not found in data.")

    vol_surface = pd.DataFrame(index=moneyness_levels)
    for matu, moneyness_data in nested_dict[date_str][udl1]['IV'].items():
        vol_surface[matu] = [
            moneyness_data.get(moneyness, np.nan) - nested_dict[date_str][udl2]['IV'][matu].get(moneyness, np.nan)
            for moneyness in moneyness_levels
        ]
    vol_surface = vol_surface.T
    vol_surface.columns = [f'{mon}' for mon in vol_surface.columns]
    vol_surface.index.name = 'Maturity'
    vol_surface = vol_surface.applymap(lambda x: round(x, 2) if pd.notna(x) else np.nan)
    return vol_surface

def plot_surface(udl1, udl2, date, surface_type, start_date=None, end_date=None):
    try:
        date = pd.Timestamp(date)

        if surface_type == 'Level':
            vol_surface = create_vol_surface(nested_dict, date, udl1, udl2, moneyness_list)
            vol_surface = ensure_numerical(vol_surface)  # Ensure numerical format
            display(style_df(vol_surface, f"Volatility Spread Surface ({udl1} - {udl2})"))
        elif surface_type == 'Percentile':
            if start_date and end_date:
                start_date = pd.Timestamp(start_date)
                end_date = pd.Timestamp(end_date)
                if start_date > end_date:
                    print("Start date cannot be after end date.")
                    return
                filtered_dict = {k: v for k, v in nested_dict.items() if start_date <= pd.Timestamp(k) <= end_date}
                percentile_surface = calculate_percentile_rank_surface(filtered_dict, udl1, udl2, end_date, moneyness_list)
                percentile_surface = ensure_numerical(percentile_surface)  # Ensure numerical format
                title = f"Percentile Spread Surface ({udl1} - {udl2}) From: {start_date.strftime('%Y-%m-%d')} to: {end_date.strftime('%Y-%m-%d')}"
                styled_df = style_df(percentile_surface, title)
                display(styled_df)
            else:
                print("Please select start and end dates for Percentile surface.")
        elif surface_type == 'Z-score':
            if start_date and end_date:
                start_date = pd.Timestamp(start_date)
                end_date = pd.Timestamp(end_date)
                if start_date > end_date:
                    print("Start date cannot be after end date.")
                    return
                filtered_dict = {k: v for k, v in nested_dict.items() if start_date <= pd.Timestamp(k) <= end_date}
                zscore_surface = calculate_zscore_surface(filtered_dict, udl1, udl2, end_date, moneyness_list)
                zscore_surface = ensure_numerical(zscore_surface)  # Ensure numerical format
                title = f"Z-score Spread Surface ({udl1} - {udl2}) From: {start_date.strftime('%Y-%m-%d')} to: {end_date.strftime('%Y-%m-%d')}"
                styled_df = style_df(zscore_surface, title)
                display(styled_df)
            else:
                print("Please select start and end dates for Z-score surface.")
        else:
            print("Invalid surface type selected.")
    except KeyError as e:
        print(f"KeyError: {e} - Ensure the selected date range is within the data's date range.")
    except Exception as e:
        print(f"An error occurred: {e}")

In [12]:
# Widgets for selecting UDLs, dates, and surface types
udl1_widget = widgets.Dropdown(options=udl_list, description='UDL1:')
udl2_widget = widgets.Dropdown(options=udl_list, description='UDL2:')
surface_type_widget = widgets.Dropdown(options=['Level', 'Percentile', 'Z-score'], description='Type:')
date_widget = widgets.DatePicker(description='Date', value=pd.to_datetime(end_date))
start_date_widget = widgets.DatePicker(description='Start Date', value=pd.to_datetime(start_date))
end_date_widget = widgets.DatePicker(description='End Date', value=pd.to_datetime(end_date))

# Observe changes in surface type to toggle date widgets
surface_type_widget.observe(lambda change: toggle_date_widgets(change['new']), names='value')

# Function to toggle date widgets visibility
def toggle_date_widgets(surface_type):
    if surface_type in ['Percentile', 'Z-score']:
        start_date_widget.layout.display = 'block'
        end_date_widget.layout.display = 'block'
        date_widget.layout.display = 'none'
    else:
        start_date_widget.layout.display = 'none'
        end_date_widget.layout.display = 'none'
        date_widget.layout.display = 'block'

# Function to ensure numerical format
def ensure_numerical(df):
    return df.apply(pd.to_numeric, errors='coerce')

# Create interactive output
output = widgets.interactive_output(plot_surface, {
    'udl1': udl1_widget, 
    'udl2': udl2_widget,
    'date': date_widget,
    'surface_type': surface_type_widget,
    'start_date': start_date_widget,
    'end_date': end_date_widget
})

# Layout for widgets
left_box = widgets.VBox([udl1_widget, udl2_widget, surface_type_widget], layout=widgets.Layout(margin='10px'))
right_box = widgets.VBox([start_date_widget, end_date_widget, date_widget], layout=widgets.Layout(margin='10px'))
top_box = widgets.HBox([left_box, right_box], layout=widgets.Layout(justify_content='space-between', align_items='center', margin='10px'))
main_box = widgets.VBox([top_box, output])

# Set initial visibility
toggle_date_widgets(surface_type_widget.value)

# Display the widgets and output
display(main_box)